In [ ]:
#DOWNLOAD NLP locally

import sys
sys.path.append('../input/nlp-repo')
!cp -r ../input/nlp-repo ./
!mv nlp-repo NLP

In [ ]:
!pip3 install --no-index fsspec==0.9.0 --find-links=file:///kaggle/input/fsspec-package/fsspec

In [ ]:
!pip3 install --no-index datasets --find-links=file:///kaggle/input/datasets-package/datasets

In [ ]:
from NLP.dataset_utils import clean_datasets
from datasets import Dataset

In [ ]:
trainset, testset = clean_datasets()

In [ ]:
trainset.features

In [ ]:
import transformers
tokenizer = transformers.RobertaTokenizer.from_pretrained("../input/roberta-transformers-pytorch/roberta-base")

In [ ]:
model = transformers.RobertaForSequenceClassification.from_pretrained("../input/roberta-transformers-pytorch/roberta-base", num_labels=1)

In [ ]:
def tokenize_function(examples, name="text"):
    return tokenizer(examples[name], padding="max_length", truncation=True)

tokenized_trainset = trainset.map(tokenize_function, batched=True)
tokenized_testset = testset.map(tokenize_function, batched=True)

In [ ]:
tokenized_trainset.features

In [ ]:
tokenized_trainset = tokenized_trainset.remove_columns(['text', 'id', 'index', 'standard_error'])
tokenized_testset = tokenized_testset.remove_columns(['text', 'id', 'index', 'standard_error'])

tokenized_trainset.set_format('torch')
tokenized_testset.set_format('torch')

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_trainset, shuffle=True, batch_size=8)
test_dataloader = DataLoader(tokenized_testset, batch_size=8)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
%%capture
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
from tqdm.auto import tqdm

#progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in tqdm(range(num_epochs)):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        #progress_bar.update(1)

In [ ]:
#Trying a submission
import pandas as pd

submission_data = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission_dataset = Dataset.from_pandas(submission_data)

#submission_dataset.features

In [ ]:
def tokenize_function(examples, name="excerpt"):
    return tokenizer(examples[name], padding="max_length", truncation=True)

sub_tokenized = submission_dataset.map(tokenize_function, batched=True)
sub_tokenized.features

In [ ]:
#need this for the submission file
ids_sub = submission_data['id']

In [ ]:
sub_tokenized = sub_tokenized.remove_columns(['excerpt', 'id', 'license', 'url_legal'])

sub_tokenized.set_format('torch')

In [ ]:
sub_dataloader = DataLoader(sub_tokenized, batch_size=7)

In [ ]:
for b in sub_dataloader:
    batch = {k: v.to(device) for k, v in b.items()}

In [ ]:
outputs = model(**batch)

In [ ]:
print(outputs)

In [ ]:
results = pd.Series([],dtype=pd.StringDtype())
for i in range(len(ids_sub)):
        results.at[i] = float(outputs[0][i].item())
print(results)

In [ ]:
s = pd.Series({'id': ids_sub, 'target': results})
s.to_csv('submission.csv')

In [ ]:
#ids_new = pd.Series( (v for v in ids_sub))
#data = [ids_new, results]

In [ ]:
"""
headers = ["id", "target"]
submission = pd.concat(data, axis=1, keys = headers)

#show submission
print(submission)

#save submission
submission.to_csv('submission.csv', index = False)
"""

In [ ]:
!rm -r ./NLP